In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from lxml import etree

In [ ]:
from typing import List, Tuple

In [ ]:
import string, re

In [126]:
%pip install spacy-udpipe

In [ ]:
import spacy_udpipe

In [ ]:
%pip install pymorphy2==0.8

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 5.4MB/s 


In [ ]:
spacy_udpipe.download("ru-syntagrus") 
nlp = spacy_udpipe.load("ru-syntagrus")

Already downloaded a model for the 'ru-syntagrus' language


In [83]:
def chewable(s):                                                # Убираем лишнее 
  lemmas = [token.lemma_ for token in nlp(s)]
  s_lemmd = ' '.join(lemmas)                                                
  webs = '(\S+\.\S+\.?\S+[\s^\v]?)'
  nums = '(\d+[\s^\v]?)'
  re_webs = re.compile(webs)                                    # Убираем сслыки
  re_pnct = re.compile('[%s]' % re.escape(string.punctuation))  # Пунктуацию
  re_nums = re.compile(nums)                                    # Почему-то числа становились наиболее значимыми признаками - убираю
  s_dewebbed = re_webs.sub('', s_lemmd)
  s_stripped = re_pnct.sub('', s_dewebbed)
  s_denummed = re_nums.sub('', s_stripped)
  return s_denummed

In [85]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = chewable(column.text)                                  #тут оно, собственно, и торчит
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [86]:
fXML = '/content/drive/My Drive/bank_train_2016.xml'

In [88]:
texts, labels = load_sentirueval_2016(fXML)

In [89]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 9392, number of labels is 9392.


In [90]:
import random

In [92]:
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

positive => Фич с покупка код для Itunes  AppStore в приложение Сбербанк онлайн для iOS реально работать
neutral => потреб кредит в втб 
neutral => Райфайзный банк аваль кредить 
negative => в список финансовый институт  попасть под волна секторальный санкция Европейский союз  войти  Газпромбанк   Сбербанк   Россельхозбанк 
negative => ну почему у Сбербанк все через жоп  почему продлить дебетовый карта можно только после декабрь   
positive => услуга туроператор за спасибо стать доступный участник программа Сбербанк из ЕАО    отдых
neutral => банк москва беспроцентный кредит 
neutral =>  radiot кредит наличный райффайзенбанк 
neutral => как скинуть деньги на телефон с карта сбербанок 
negative => Банкомат Сбербанк в Москва прекратить прием пятитысячный купюра изза фальшивка  желтый
negative => на пройти неделя США расширить санкция в отношение Россия  к список добавить Газпромбанк  Внешэкономбанк   Роснефть  и НОВАТЭК 
positive => Сбербанк снижать ставка на ипотека 
neutral => Карельск

In [93]:
positive_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'positive']
negative_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'negative']

In [94]:
for cur in positive_tweets[:5]: print(cur)

sawikshuster YevhenS главный чтобы банк СБЕР и ВТБ   
самый выгодный автокредит в втб 
легко можно получить денежный кредит ы вт банк
снижение процентный ставка по кредит на недвижимость сбербанк 
в наш сбербанк прийти американец и просить сфоткаться с он у банкомат  D я васто быть в шок и на фотка полюбому получиться смешной xD


In [95]:
for cur in negative_tweets[:5]: print(cur)

ShtirliZ Zhukovaolga winzard tereshenkov обязательно про сбербанк писать  временами хлестко интерать магазин финт выкручивать 
UmkaAlfariusigorsechin втб и сбер точно  вопрос  блокировка операция или запрет на кредит 
Канада ввести санкция против Газпромбанк  ВЭБ  Новатэк и российский оборонный компания
RT letokot  если Сбер и ВТБ попасть под санкция  быть жоп
RT tvru   Канада ввести санкция против оборонный и сырьевой компания и банк РФ  среди они  в частность    Газпромбанк     Внешэк …


In [96]:
from nltk import word_tokenize

In [97]:
from sklearn.feature_extraction.text import CountVectorizer

In [98]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize)

In [99]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [100]:
vectorizer.fit(texts)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x7f97f1528d90>,
                vocabulary=None)

In [101]:
print(vectorizer.get_feature_names()[0:20])

['a', 'abaradagaнлат', 'aberadez', 'abetaa', 'abncapital', 'abramovdeputat', 'abunyasha', 'abysov', 'adagamov', 'adairrasi', 'adaron', 'aday', 'adenisenko', 'adindex', 'adlarguan', 'admeru', 'admitad', 'adubter', 'adworningju', 'aeroflot']


In [102]:
print(len(vectorizer.get_feature_names()))

8698


In [103]:
X = vectorizer.transform(texts)

In [104]:
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [105]:
print(texts[0])

брать кредит тюмень альфа банк


In [106]:
print(X[0])

  (0, 1815)	1
  (0, 1965)	1
  (0, 2144)	1
  (0, 4118)	1
  (0, 7898)	1


In [107]:
print(vectorizer.get_feature_names()[6321])

пришлть


In [108]:
print(vectorizer.get_feature_names()[8000])

укрсиббанк


In [109]:
from sklearn.feature_extraction.text import TfidfTransformer

In [110]:
transformer = TfidfTransformer().fit(X)

In [111]:
X_transformed = transformer.transform(X)

In [112]:
print(X_transformed[0])

  (0, 7898)	0.7639100517232897
  (0, 4118)	0.2173288930139871
  (0, 2144)	0.4039888226437941
  (0, 1965)	0.24576972473891776
  (0, 1815)	0.38157549543710667


In [113]:
print(vectorizer.get_feature_names()[7199])

скупка


In [114]:
tokens_with_IDF = list(zip(vectorizer.get_feature_names(), transformer.idf_))

In [115]:
for feature, idf in tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.049108 => a
9.454573 => abaradagaнлат
9.454573 => aberadez
9.454573 => abetaa
9.454573 => abncapital
9.454573 => abramovdeputat
9.454573 => abunyasha
9.454573 => abysov
9.454573 => adagamov
9.454573 => adairrasi
9.454573 => adaron
9.454573 => aday
9.454573 => adenisenko
9.454573 => adindex
9.454573 => adlarguan
9.454573 => admeru
9.454573 => admitad
9.454573 => adubter
9.454573 => adworningju
8.538282 => aeroflot


In [116]:
sorted_tokens_with_IDF = sorted(tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [117]:
for feature, idf in sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => abaradagaнлат
9.454573 => aberadez
9.454573 => abetaa
9.454573 => abncapital
9.454573 => abramovdeputat
9.454573 => abunyasha
9.454573 => abysov
9.454573 => adagamov
9.454573 => adairrasi
9.454573 => adaron
9.454573 => aday
9.454573 => adenisenko
9.454573 => adindex
9.454573 => adlarguan
9.454573 => admeru
9.454573 => admitad
9.454573 => adubter
9.454573 => adworningju
9.454573 => afyanik
9.454573 => agatacrysty


In [118]:
from sklearn.feature_selection import SelectPercentile, chi2

In [119]:
selector = SelectPercentile(chi2, percentile=20)

In [120]:
selector.fit(X_transformed, labels)

SelectPercentile(percentile=20, score_func=<function chi2 at 0x7f97dfc5f730>)

In [121]:
selected_tokens_with_IDF = [tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]

In [122]:
print(len(selected_tokens_with_IDF))

1740


In [123]:
for feature, idf in selected_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => aberadez
9.454573 => abunyasha
9.454573 => adworningju
8.761426 => akselenc
9.454573 => alexigrey
4.905973 => alfabank
6.592372 => alfabankby
9.454573 => alushin
9.454573 => andreymalgin
9.454573 => andreytr
8.761426 => antimaydan
9.049108 => apple
9.454573 => appstore
9.454573 => arhangelmisha
9.454573 => artemycheros
8.761426 => awards
9.049108 => balamot
9.454573 => bankitb
8.538282 => bbb
9.454573 => behlbride


In [124]:
selected_and_sorted_tokens_with_IDF = sorted(selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [125]:
for feature, idf in selected_and_sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => aberadez
9.454573 => abunyasha
9.454573 => adworningju
9.454573 => alexigrey
9.454573 => alushin
9.454573 => andreymalgin
9.454573 => andreytr
9.454573 => appstore
9.454573 => arhangelmisha
9.454573 => artemycheros
9.454573 => bankitb
9.454573 => behlbride
9.454573 => bosacoworavu
9.454573 => bullethunter
9.454573 => buribedixuk
9.454573 => catspower
9.454573 => cgillenpiy
9.454573 => chumikov
9.454573 => citybusicyti
9.454573 => come
